In [1]:
import requests
import json
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
key = '?api_key=5e34fafe-43b3-4c41-8c0c-22d1c57b4e7b'

In [3]:
"""
publicMatches = 'https://api.opendota.com/api/publicMatches?less_than_match_id=6100477607'# + str()
r = requests.get(publicMatches)
testdata = json.loads(r.text)
testdata
"""

"\npublicMatches = 'https://api.opendota.com/api/publicMatches?less_than_match_id=6100477607'# + str()\nr = requests.get(publicMatches)\ntestdata = json.loads(r.text)\ntestdata\n"

In [4]:
# 6011384988 - May 25 match near start of patch 7.29d - https://dota2.fandom.com/wiki/Patches#2021

In [5]:
# test to see requests
# run 1 - start = 6119181964 error at 6075290113
# run 2 - start = 6075290113

start = 6075290113
end = 6011384988
data = []
while True:
    link = 'https://api.opendota.com/api/publicMatches' + str(key) + '&less_than_match_id=' + str(start) 
    r = requests.get(link)
    cur_data = json.loads(r.text)
    if type(cur_data)is not dict: 
        data = data + cur_data    
    else: # Error handling
        print(cur_data)
        break
    #seq = [x['match_id'] for x in data]
    #print("Cur Max: ", max(seq), "Cur Min: ", min(seq))
    start = data[-1].get('match_id')
    if start < end: # Check if reached beginning of patch (roughly)
        print("End reached", start)
        break

End reached 6011384107


In [14]:
#print(6119181964- 6119003515)
df = pd.DataFrame(data)
df

,match_id,match_seq_num,radiant_win,start_time,duration,avg_mmr,num_mmr,lobby_type,game_mode,avg_rank_tier,num_rank_tier,cluster,radiant_team,dire_team
0,6075290019,5086093418,True,1625515665,2378,3334.0,3.0,7,22,36,7,187,"14,74,129,68,10","94,91,34,35,69"
1,6075290018,5086093129,False,1625515665,2394,NaN,NaN,7,22,21,2,202,"81,86,39,84,62","68,47,1,121,59"
2,6075290017,5086084591,True,1625515660,1752,3417.0,1.0,7,22,53,4,193,"68,99,104,20,93","54,50,23,11,111"
3,6075290016,5086095428,True,1625515654,2423,2485.0,1.0,7,22,37,3,201,"26,8,67,10,51","35,29,119,41,14"
4,6075290015,5086078250,True,1625515649,1442,NaN,NaN,0,22,28,2,241,"39,50,99,18,100","109,25,12,79,7"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3808894,6011384113,5035294458,False,1621984962,2108,NaN,NaN,7,22,12,2,181,"109,29,27,46,42","59,84,121,36,44"
3808895,6011384112,5035292430,False,1621984957,1843,NaN,NaN,7,22,39,2,251,"22,112,6,123,14","46,7,93,27,13"
3808896,6011384111,5035296717,False,1621984957,2428,3039.0,3.0,7,22,52,7,186,"26,98,40,93,86","50,35,12,7,99"
3808897,6011384108,5035291897,True,1621984947,1759,3819.0,4.0,7,22,46,7,251,"17,36,41,121,19","53,1,13,49,14"


In [15]:
# Checking for duplicates
df['match_id'].value_counts().describe()

count    3808899.0
mean           1.0
std            0.0
min            1.0
25%            1.0
50%            1.0
75%            1.0
max            1.0
Name: match_id, dtype: float64

In [16]:
df['lobby_type'].value_counts()

7    3007560
0     794978
9       4458
1       1903
Name: lobby_type, dtype: int64

In [17]:
"""
https://github.com/odota/dotaconstants/blob/master/json/lobby_type.json

Drop irrelevant lobby_types
Irrelevant: 
1 = lobby_type_practice
Relevant: 
0 = lobby_type_normal
7 = lobby_type_ranked
9 = lobby_type_battle_cup
"""
df.drop(df[df['lobby_type'] == 1].index, inplace = True)
df['lobby_type'].value_counts()

7    3007560
0     794978
9       4458
Name: lobby_type, dtype: int64

In [18]:
df['game_mode'].value_counts()

22    3401234
3      257088
4      136437
2        6173
5        6064
Name: game_mode, dtype: int64

In [19]:
"""
https://github.com/odota/dotaconstants/blob/master/json/game_mode.json
Drop irrelevant game_modes
Irrelevant: 
1 = lobby_type_practice
2 = game_mode_captains_mode
3 = game_mode_random_draft
4 = game_mode_single_draft
5 = game_mode_all_random
Relevant: 
16 = game_mode_captains_draft
22 = game_mode_all_draft
"""
todrop = [1,2,3,4,5]
for i in todrop:
    df.drop(df[df['game_mode'] == i].index, inplace = True)
df['game_mode'].value_counts()

22    3401234
Name: game_mode, dtype: int64

In [20]:
## This is to export the dataframe to csv file
df.to_csv(index=False)
compression_opts = dict(method='zip', archive_name='data.csv')  
df.to_csv('data.zip', index=False,
          compression=compression_opts)


In [21]:
df

,match_id,match_seq_num,radiant_win,start_time,duration,avg_mmr,num_mmr,lobby_type,game_mode,avg_rank_tier,num_rank_tier,cluster,radiant_team,dire_team
0,6075290019,5086093418,True,1625515665,2378,3334.0,3.0,7,22,36,7,187,"14,74,129,68,10","94,91,34,35,69"
1,6075290018,5086093129,False,1625515665,2394,NaN,NaN,7,22,21,2,202,"81,86,39,84,62","68,47,1,121,59"
2,6075290017,5086084591,True,1625515660,1752,3417.0,1.0,7,22,53,4,193,"68,99,104,20,93","54,50,23,11,111"
3,6075290016,5086095428,True,1625515654,2423,2485.0,1.0,7,22,37,3,201,"26,8,67,10,51","35,29,119,41,14"
4,6075290015,5086078250,True,1625515649,1442,NaN,NaN,0,22,28,2,241,"39,50,99,18,100","109,25,12,79,7"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3808894,6011384113,5035294458,False,1621984962,2108,NaN,NaN,7,22,12,2,181,"109,29,27,46,42","59,84,121,36,44"
3808895,6011384112,5035292430,False,1621984957,1843,NaN,NaN,7,22,39,2,251,"22,112,6,123,14","46,7,93,27,13"
3808896,6011384111,5035296717,False,1621984957,2428,3039.0,3.0,7,22,52,7,186,"26,98,40,93,86","50,35,12,7,99"
3808897,6011384108,5035291897,True,1621984947,1759,3819.0,4.0,7,22,46,7,251,"17,36,41,121,19","53,1,13,49,14"
